# Demo 2: Bayesian Integration

In Demo 2, we will seek to better understand optimal Bayesian Integration.
The demo is split in two different experimental scenarios, which are explored in the second half of the lecture.
Each demo will be accompanied by **guiding questions**, which will help with the learning process as we go.
    
**Part A** illustrates a unisensory (proprioceptive) hand localization experiment with sensory input and a prior.

**Part B** illustrates a multisensory (proprioceptive & vision) hand localization experiment with sensory input and a prior.



## Demo 2A: Proprioceptive Hand Localization

In the current demo, we will explore Bayesian integration in the case of proprioceptive hand localization, 
where the perceived hand location (posterior), is derived from integrating proprioceptive input (likelihood)
and a prior distribution for hand location.

In the demo, the likelihood is illustrated with a red curve, the prior with a blue curve, and the posterior with a purple curve.

Two adjustable settings control the properties of the Likelihood distribution.

1. "P Stimulus" controls the location of the hand.
2. "P Noise σ" controls the proprioceptive noise.

Two adjustable settings control the properties of the Prior distribution.

1. "Prior" controls the mean of the prior distribution.
2. "Prior σ" controls the standard deviation of the prior distribution.

To make a measurement, click the box "Make Measurement". 
Doing so will randomly sample a sensory observation from the measurement distribution and illustrate the Likelihood distribution.

To derive a posterior, click the box "Show Posterior". 


### Guiding Questions
1. Let's first explore how the likelihood and the prior vary from measurement-to-measurement. Make several measurements using the "Make Measurement" checkbox. 
How does taking a measurement affect the likelihood and the prior? What about if you change the mean locations and the noise/uncertainty levels?

2. Click to show the posterior distribution. What do you notice about it compared to the likelihood and the prior? 
If you change the noise and uncertainty levels, what does this do to the posterior? Explore several combinations.

3. How does changing the distance between the likelihood and prior means change the size of the bias?
                               

In [15]:
"""
DEMO 2: Full Bayesian Integration in two experimental scenarios

DEMO 2A: Scenario 1: Proprioceptive localization with a prior
DEMO 2B: Scenario 2: Visuo-proprioceptive localization without a prior

"""

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, Checkbox, VBox, HBox
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)

P_RANGE = (-10, 10)
SIGMA_P_RANGE = (1, 5)

PR_RANGE = (-10, 10)
SIGMA_PR_RANGE = (1, 10)


XLIM = (-20, 20)   # (-17, 17)
YLIM = (-0.002, 0.05)         # room for baseline + PDF peak (0.15)


def full_bayes(s_p: float = 0.0, sigma_p: float = 1.0,
                     mu_pr: float = 0.0, sigma_pr: float = 1.0,
                     make_m: bool = True, show_post: bool=True):
    
    
    fig, ax = plt.subplots(figsize=(8, 6))

    xs = np.linspace(XLIM[0], XLIM[1], 400) # centers of measurements

    ax.axvline(s_p, color="grey", linestyle="--")

    # Prior
    var_pr = sigma_pr**2
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr)) # unormalized prior distribution

    norm_pr = ((1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr))).sum() # normalization constant for prior
    pdf_pr_norm = pdf_pr / norm_pr # normalized prior distribution

    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)

    
    # Proprioceptive measurement
    if make_m:
        rng = np.random.default_rng()
        noise = rng.normal(loc=0, scale=sigma_p, size=1)            
        x_obs_p = s_p + noise

        
    
    
        # Proprioceptive likelihood
        var_p = sigma_p**2
        pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p - xs)** 2 / var_p)) # unormalized likelihood
    
        norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)** 2 / var_p))).sum() # normalization constant for P
        pdf_p_norm = pdf_p / norm_p # normalized proprioceptive likelihood
    
        ax.plot(xs, pdf_p_norm, color="red", linewidth=2)


    

        if show_post:
            pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p-xs)** 2 / var_p))
            pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr)) # unormalized prior distribution
            pdf_post = pdf_p*pdf_pr
            pdf_post_norm = pdf_post / pdf_post.sum() # normalized posterior
            ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)


    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45*XLIM[1],0.9*YLIM[1],'Prior Dist.',color="blue",fontsize=12)
    ax.text(0.45*XLIM[1],0.83*YLIM[1],'Likelihood Dist.',color="red",fontsize=12)
    ax.text(0.45*XLIM[1],0.76*YLIM[1],'Posterior Dist.',color="purple",fontsize=12)


    # Aesthetics
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "s_p": FloatSlider(value=-7.0, min=P_RANGE[0], max=P_RANGE[1], step=0.1,
                       description="P Stimulus:", continuous_update=False,
                       readout_format=".1f"),
    "sigma_p": FloatSlider(value=3.0, min=SIGMA_P_RANGE[0], max=SIGMA_P_RANGE[1], step=0.1,
                          description="P Noise σ:", continuous_update=False),
    "mu_pr": FloatSlider(value=0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                       description="Prior:", continuous_update=False,
                       readout_format=".1f"),
    "sigma_pr": FloatSlider(value=6.0, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                          description="Prior σ:", continuous_update=False),
    "show_post": Checkbox(value=False, description="Show Posterior"),

    "make_m": Checkbox(value=False, description="Make Measurement"),

}

interactive_plot = interactive(full_bayes, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["s_p"], controls["sigma_p"],controls["mu_pr"], controls["sigma_pr"],controls["show_post"],
                controls["make_m"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()


## Demo 2B: Multisensory Hand Localization

In the current demo, we will explore Bayesian integration in the case of multisensory proprioception+visual hand localization, 
where the perceived hand location (posterior), is derived from integrating two likelihoods, one for proprioceptive input and 
the other for visual input; a prior can also be added

In the demo, the proprioceptive likelihood is illustrated with a red curve, the visual likelihod is illustrated with a green curve
the prior with a blue curve, and the posterior with a purple curve.

Two adjustable settings control the properties of the Proprioceptive Likelihood distribution.

1. "P Stimulus" controls the location of the hand, sensed via proprioception.
2. "P Noise σ" controls the proprioceptive noise.

Two adjustable settings control the properties of the Visual Likelihood distribution.

1. "V Stimulus" controls the location of the hand, sensed via vision.
2. "V Noise σ" controls the visual noise.

To make a measurement, click the box "Make Measurement". 
Doing so will randomly sample a sensory observation from the measurement distributions and illustrate the Likelihood distributions. 

To illustrate the posterior, click the box "Show Posterior". 

You can also add a prior distribution (click "Add a prior"). The prior mean is set to 0 and the standard deviation is set to 5.


### Guiding Questions
1. Let's first explore what varies from measurement to measurement. Make several measurements. How does this affect each likelihood? 
What about if you change the mean locations and the noise/uncertainty levels?

2. Click to show the posterior distribution. What do you notice about it compared to each likelihood? 
If you change the noise levels, what does this do to the posterior? Explore several combinations, including changing one
    cue at a time. How does the posterior relate to each senosry likelihood?
                               
3. Finally, let's explore what happens if we add a prior. Check this option in the menu. How does change the posterior?
What if you modify the noise levels and stimulus locations?

In [16]:
# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)
V_RANGE = (-10, 10)          # from sliders
SIGMA_V_RANGE = (1, 5)    # from sliders

def vp_integration(s_v: float = 0.0, sigma_v: float = 1.0,
                     s_p: float = 0.0, sigma_p: float = 1.0,
                     make_m: bool = True,
                     show_post: bool = True,
                     add_prior: bool=True):


    fig, ax = plt.subplots(figsize=(8, 6))


    if make_m:
        # Take measurements
        xs = np.linspace(XLIM[0], XLIM[1], 400) # centers of measurements
        rng = np.random.default_rng()
        x_obs_v = rng.normal(loc=s_v, scale=sigma_v, size=1)
        x_obs_p = rng.normal(loc=s_p, scale=sigma_p, size=1)
        
    
    
        # Visual Likelihood
        var_v = sigma_v**2
        norm_v = ((1 / (np.sqrt(2 * np.pi) * var_v)) * np.exp(-0.5 * ((0 - xs)** 2 / var_v))).sum()
        pdf_v = (1 / (np.sqrt(2 * np.pi) * var_v)) * np.exp(-0.5 * ((x_obs_v - xs)** 2 / var_v))
        pdf_v_norm = pdf_v / norm_v 
        ax.plot(xs, pdf_v_norm, color="green", linewidth=2)
        
    
    
    
           
        # Proprioceptive Likelihood
        var_p = sigma_p**2
        norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)** 2 / var_p))).sum()
        pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p - xs)** 2 / var_p) )
        pdf_p_norm = pdf_p / norm_p
        ax.plot(xs, pdf_p_norm, color="red", linewidth=2)

        if add_prior:
            sigma_pr = 5
            mu_pr = 0
            var_pr = sigma_pr**2
            pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr)) # unormalized prior distribution
        
            norm_pr = ((1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr))).sum() # normalization constant for prior
            pdf_pr_norm = pdf_pr / norm_pr # normalized prior distribution
        
            ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)
                        
        if show_post: 
            if add_prior:
               
            # Posterior
        
                pdf_post = pdf_p*pdf_v*pdf_pr
        
            else:
                pdf_post = pdf_p*pdf_v
            
            pdf_post_norm = pdf_post / pdf_post.sum()
            ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)




        
    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45*XLIM[1],0.9*YLIM[1],'P Likelihood Dist.',color="red",fontsize=12)
    ax.text(0.45*XLIM[1],0.83*YLIM[1],'V Likelihood Dist.',color="green",fontsize=12)
    ax.text(0.45*XLIM[1],0.76*YLIM[1],'Posterior Dist.',color="purple",fontsize=12)
    ax.text(0.45*XLIM[1],0.69*YLIM[1],'Prior Dist.',color="blue",fontsize=12)

    
    # Aesthetics
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "s_v": FloatSlider(value=5.0, min=V_RANGE[0], max=V_RANGE[1], step=0.1,
                       description="V Stimulus:", continuous_update=False,
                       readout_format=".1f"),
    "sigma_v": FloatSlider(value=2.5, min=SIGMA_V_RANGE[0], max=SIGMA_V_RANGE[1], step=0.1,
                          description="V Noise σ:", continuous_update=False),
    "s_p": FloatSlider(value=-7.0, min=P_RANGE[0], max=P_RANGE[1], step=0.1,
                       description="P Stimulus:", continuous_update=False,
                       readout_format=".1f"),
    "sigma_p": FloatSlider(value=4.0, min=SIGMA_P_RANGE[0], max=SIGMA_P_RANGE[1], step=0.1,
                          description="P Noise σ:", continuous_update=False),
    "make_m": Checkbox(value=True, description="Make Measurement"),
    "show_post": Checkbox(value=False, description="Show Posterior"),
    "add_prior": Checkbox(value=False, description="Add a prior"),

}

interactive_plot = interactive(vp_integration, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["s_p"], controls["sigma_p"],controls["s_v"], controls["sigma_v"],
                controls["make_m"],controls["show_post"],controls["add_prior"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()
